# Sunshine Hours

In [1]:
from math import pi

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
shift = 15
target_cities = ["Barcelona", "New York City", "Lyon", "Paris", "Amsterdam", "Berlin"]
angles = {
    "JAN": -shift, "FEB": -30 - shift, "MAR": -60 - shift, "APR": 90 - shift, "MAY": 60 - shift, "JUN": 30 - shift,
    "JUL": -shift, "AUG": -30 - shift, "SEP": -60 - shift, "OCT": 90 - shift, "NOV": 60 - shift, "DEC": 30 - shift,
}

In [4]:
def get_data(cities):
    import pandas as pd
    raw_df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/sunshine_hours.csv")
    filtered_df = raw_df[raw_df["City"].isin(cities)]
    df = pd.melt(filtered_df, id_vars=["Country", "City"], value_vars=filtered_df.columns[2:-1], \
                 var_name="Month", value_name="Sunshine hours")
    df["Month"] = df["Month"].str.upper()
    df["Angle"] = df["Month"].apply(lambda m: angles[m])
    return df.sort_values(by=["City"], key=lambda d: [target_cities.index(v) for i, v in d.items()]).reset_index(drop=True)

In [5]:
df = get_data(target_cities)
print(df.shape)
df.head(3)

(72, 5)


,Country,City,Month,Sunshine hours,Angle
0,Spain,Barcelona,AUG,264.0,-45
1,Spain,Barcelona,MAY,258.0,45
2,Spain,Barcelona,MAR,206.0,-75


In [6]:
font_family = "ParaType"
font_face = 'bold'
background_color = "#fcf3e4"
grid_color = "#ccc2a2"

y_breaks = [100, 200]
y_breaks_data = {
    "x": ["JAN"] * len(y_breaks),
    "y": y_breaks,
    "text": [str(b) for b in y_breaks],
}

ggplot(df, aes(as_discrete("Month", levels=list(angles.keys())), "Sunshine hours")) + \
    geom_text(aes("x", "y", label="text"), data=y_breaks_data, \
              angle=-shift, family=font_family, fontface=font_face, size=6, vjust=0) + \
    geom_bar(stat='identity', color="black", fill="#fbc117", \
             tooltips=layer_tooltips().title("@City").line("@Month: @{Sunshine hours}")) + \
    geom_label(aes("Month", label="Month", angle="Angle"), y=330, \
               family=font_family, fontface=font_face, size=8, \
               label_padding=0, label_r=0, label_size=0, fill=background_color) + \
    facet_wrap(facets="City", order=None) + \
    coord_polar(start=pi*shift/180) + \
    labs(title="Comparing Sunshine Hours", \
         caption="Sunshine duration is expressed in (average) hours per month\n"
                 "Data source: "
                   "<a href=\"https://www.kaggle.com/datasets/bilalwaseer/sunshine-hours-for-cities-around-the-world\">"
                     "sunshine hours for cities around the world"
                   "</a>\n"
                 "Original plot: "
                    "<a href=\"https://www.linkedin.com/feed/update/urn:li:activity:7186216546602524672/\">"
                      "Giulia Mezzadri, PhD's Post"
                    "</a>") + \
    ggsize(1000, 850) + \
    theme(axis='blank', \
          plot_title=element_text(size=32, family=font_family, face=font_face, margin=[20, 20, 30]), \
          plot_caption=element_text(family=font_family, face=font_face), \
          strip_text=element_text(size=20, family=font_family, face=font_face), \
          plot_background=element_rect(fill=background_color), \
          panel_grid=element_line(color=grid_color, size=2))